# Tensorflow low level 

In [1]:
import tensorflow as tf

In [56]:
# defines a matrix kinda like np.array([])
t = tf.constant([[1.,2.,3.], [4.,5.,6.]])

In [57]:
t.shape

TensorShape([2, 3])

In [58]:
# defines scalar 
tf.constant(42)

<tf.Tensor: id=119, shape=(), dtype=int32, numpy=42>

In [59]:
# indexing works like Numpy: row then col 
t[:, 2:]

<tf.Tensor: id=123, shape=(2, 1), dtype=float32, numpy=
array([[3.],
       [6.]], dtype=float32)>

In [60]:
# tensor operation 
t + 10
tf.square(t)
t @ tf.transpose(t) # matrix multiplication equal to tf.matmul()
tf.matmul(t, tf.transpose(t))

<tf.Tensor: id=132, shape=(2, 2), dtype=float32, numpy=
array([[14., 32.],
       [32., 77.]], dtype=float32)>

## Tensor and Numpy 

Notice that NumPy uses 64-bit precision by default, while TensorFlow uses 32-bit. This is because 32-bit precision is generally more than enough for neural networks, plus it runs faster and uses less RAM. So when you create a tensor from a NumPy array, make sure to set dtype=tf.float32.

In [61]:
import numpy as np

In [62]:
# convert numpy to tensor 
a = np.array([2.,4.,5.])
tf.constant(a)

<tf.Tensor: id=133, shape=(3,), dtype=float64, numpy=array([2., 4., 5.])>

In [63]:
# convert tensor to numpy
t.numpy()

array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)

In [64]:
# apply tensor OP directly to np.array
tf.square(a)

<tf.Tensor: id=135, shape=(3,), dtype=float64, numpy=array([ 4., 16., 25.])>

In [65]:
# apply numpy OP directly to a tensor 
np.square(t)

array([[ 1.,  4.,  9.],
       [16., 25., 36.]], dtype=float32)

## tf.constant and tf.variable

Main differences: tf.tensor create from tf.constant so far are immutable: you cannot modify them. This means that we cannot use regular tensors to implement weights in a neural network, since they need to be tweaked by backpropagation. Plus, other parameters may also need to change over time. Thus we need a tf.Variable

In [66]:
v = tf.Variable([[1., 2., 3.], [4., 5., 6.]])
v

<tf.Variable 'Variable:0' shape=(2, 3) dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

In [67]:
# tf_variable can be modified using assign() method
v.assign(2 * v)           # => [[2., 4., 6.], [8., 10., 12.]]
v[0, 1].assign(42)        # => [[2., 42., 6.], [8., 10., 12.]]
v[:, 2].assign([0., 1.])  # => [[2., 42., 0.], [8., 10., 1.]]
v.scatter_nd_update(indices=[[0, 0], [1, 2]], updates=[100., 200.])
                          # => [[100., 42., 0.], [8., 10., 200.]]

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[100.,  42.,   0.],
       [  8.,  10., 200.]], dtype=float32)>

## Other Data Structures

1) Sparse tensors (tf.SparseTensor)

    Efficiently represent tensors containing mostly zeros. The tf.sparse package contains operations for sparse tensors.

2) Tensor arrays (tf.TensorArray)

    Are lists of tensors. They have a fixed size by default but can optionally be made dynamic. All tensors they contain must have the same shape and data type.

3) Ragged tensors (tf.RaggedTensor)

    Represent static lists of lists of tensors, where every tensor has the same shape and data type. The tf.ragged package contains operations for ragged tensors.

4) String tensors

    Are regular tensors of type tf.string. These represent byte strings, not Unicode strings, so if you create a string tensor using a Unicode string (e.g., a regular Python 3 string like "café"), then it will get encoded to UTF-8 automatically (e.g., b"caf\xc3\xa9"). Alternatively, you can represent Unicode strings using tensors of type tf.int32, where each item represents a Unicode code point (e.g., [99, 97, 102, 233]). The tf.strings package (with an s) contains ops for byte strings and Unicode strings (and to convert one into the other). It’s important to note that a tf.string is atomic, meaning that its length does not appear in the tensor’s shape. Once you convert it to a Unicode tensor (i.e., a tensor of type tf.int32 holding Unicode code points), the length appears in the shape.

5) Sets

    Are represented as regular tensors (or sparse tensors). For example, tf.constant([[1, 2], [3, 4]]) represents the two sets {1, 2} and {3, 4}. More generally, each set is represented by a vector in the tensor’s last axis. You can manipulate sets using operations from the tf.sets package.

6) Queues

    Store tensors across multiple steps. TensorFlow offers various kinds of queues: simple First In, First Out (FIFO) queues (FIFOQueue), queues that can prioritize some items (PriorityQueue), shuffle their items (RandomShuffleQueue), and batch items of different shapes by padding (PaddingFIFOQueue). These classes are all in the tf.queue package.

# Keras low level API

In [68]:
# Keras low level api 
from tensorflow import keras

In [69]:
K = keras.backend

In [70]:
# all keras OP is kinda like tensorflow. 
# we use keras backend when we want to export our code to keras
K.square(K.transpose(t)) + 10

<tf.Tensor: id=166, shape=(3, 2), dtype=float32, numpy=
array([[11., 26.],
       [14., 35.],
       [19., 46.]], dtype=float32)>

# Custom Loss Functions

In [71]:
# implement Huber Loss 
def huber_fn(y_true, y_pred):
    error = y_true - y_pred
    is_small_error = tf.abs(error) < 1
    squared_loss = tf.square(error) / 2
    linear_loss  = tf.abs(error) - 0.5
    return tf.where(is_small_error, squared_loss, linear_loss)

In [72]:
# add to your keras model
# model.compile(loss=huber_fn, optimizer="nadam")
# model.fit(X_train, y_train, [...])

# Saving and Loading Models That Contain Custom Components

In [73]:
# model = keras.models.load_model("my_model_with_a_custom_loss.h5",
#                                 custom_objects={"huber_fn": huber_fn})

In [3]:
pwd

'/home/coderschool/deploying/mnist_fansipan/model_train'

In [4]:
model = tf.keras.models.load_model('./saved_model/my_model')

In [5]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_15 (Conv2D)           (None, 28, 28, 12)        108       
_________________________________________________________________
batch_normalization_20 (Batc (None, 28, 28, 12)        36        
_________________________________________________________________
activation_20 (Activation)   (None, 28, 28, 12)        0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 14, 14, 24)        10368     
_________________________________________________________________
batch_normalization_21 (Batc (None, 14, 14, 24)        72        
_________________________________________________________________
activation_21 (Activation)   (None, 14, 14, 24)        0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 7, 7, 32)         

In [ ]:
# Preprocessor layer 
class Preprocessor(keras.layers.Layer):
    def preprocess(self, data): 
        img_raw = data
        image = tf.image.decode_jpeg(img_raw, channels=1)
        image = tf.image.resize(image, [28, 28])
        image = (255 - image) / 255.0  # normalize to [0,1] range
        image = tf.reshape(image, (1, 28, 28, 1))
    
        return image

In [ ]:
# regular_inputs = keras.layers.Input(shape=[8])
# categories = keras.layers.Input(shape=[], dtype=tf.string)
# cat_indices = keras.layers.Lambda(lambda cats: table.lookup(cats))(categories)
# cat_embed = keras.layers.Embedding(input_dim=6, output_dim=2)(cat_indices)
# encoded_inputs = keras.layers.concatenate([regular_inputs, cat_embed])
# outputs = keras.layers.Dense(1)(encoded_inputs)
# model = keras.models.Model(inputs=[regular_inputs, categories],
#                            outputs=[outputs])

In [90]:
# # tf1 code 
# import os
# import tensorflow as tf

# string_inp = tf.compat.v1.placeholder(tf.string, shape=(None,))
# imgs_map = tf.map_fn(
#     tf.image.decode_image,
#     string_inp,
#     dtype=tf.uint8
# )
# imgs_map.set_shape((None, None, None, 3))
# imgs = tf.image.resize_images(imgs_map, [28, 28])
# imgs = tf.reshape(imgs, (-1, 28, 28, 3))
# img_float = tf.cast(imgs, dtype=tf.float32) / 255

# model = tf.keras.models.load_model('../saved_model/my_model')
# output = model(img_float)

# # saved model
# version = 2
# export_path = os.path.join('classifier', str(version))
# tf.saved_model.simple_save(
#     tf.keras.backend.get_session(),
#     export_path,
#     inputs={'input_image': string_inp},
#     outputs={'predictions': output})